# NPS survey in EDSL
This notebook provides sample [EDSL](https://github.com/expectedparrot/edsl) code for simulating a Net Promoter Score (NPS) survey with AI agents and large language models.
In the steps below we show how to construct an EDSL survey, create personas for AI agents to answer the questions, and then administer the survey to them. 
We also demonstrate some built-in [methods for inspecting and analyzing the dataset of results](https://docs.expectedparrot.com/en/latest/results.html) that is generated when an EDSL survey is run.

The following questions are used in the sample survey:

***On a scale from 0-10, how likely are you to recommend our company to a friend or colleague?*** <br>
*(0=Not at all likely, 10=Very likely)* <br>
*Please tell us why you gave a rating.*

***How satisfied are you with the following experience with our company?*** <br>
*Product quality* <br>
*Customer support* <br>
*Purchasing experience* <br>

***Is there anything specific that our company can do to improve your experience?***

## Technical setup
Before running the code below, ensure that you have (1) [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and (2) created a [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) to activate [remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) or stored your own [API keys for language models](https://docs.expectedparrot.com/en/latest/api_keys.html) that you want to use with EDSL. Please also see our [tutorials and documentation page](https://docs.expectedparrot.com/en/latest/index.html) on getting started using the EDSL library.

## Constructing questions
We start by selecting appropriate question types for the above questions. [EDSL comes with a variety of common question types](https://docs.expectedparrot.com/en/latest/questions.html) that we can choose from based on the form of the response that we want to get back from the model. The first quesiton is linear scale; we import the class type and then construct a question in the relevant template:

In [1]:
from edsl import QuestionLinearScale

In [2]:
q_recommend = QuestionLinearScale(
    question_name = "recommend",
    question_text = "On a scale from 0-10, how likely are you to recommend our company to a friend or colleague?",
    question_options = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    option_labels = {0:"Not at all likely", 10:"Very likely"}
)

Each question type other than free text automatically includes a "comment" field for the model to provide commentary on its response to the main question. When we run the survey, we can check that it has effectively captured the follow-on question from above--*Please tell us why you gave a rating*--and modify or add questions as needed.

For the next question, we use a `{{ placeholder }}` for an "experience" that we will insert when repeating the base question:

In [3]:
from edsl import QuestionMultipleChoice

In [4]:
q_satisfied = QuestionMultipleChoice(
    question_name = "satisfied",
    question_text = "How satisfied are you with the following experience with our company: {{ experience }}",
    question_options = [
        "Extremely satisfied",
        "Moderately satisfied",
        "Neither satisfied nor dissatisfied",
        "Moderately dissatisfied",
        "Extremely dissatisfied"
    ]
)

The third question is a simple free text question that we can choose whether to administer once or individually for each "experience" question. In the steps that follow we show how to apply survey logic to achieve this effect:

In [5]:
from edsl import QuestionFreeText

In [6]:
q_improve = QuestionFreeText(
    question_name = "improve",
    question_text = "Is there anything specific that our company can do to improve your experience?"
)

## Creating variants of questions with scenarios
Next we want to create a version of the "satisfied" question for each "experience". This can be done with `Scenario` objects--dictionaries of key/value pairs representing the content to be added to questions. Scenarios can be automatically generated from a variety of data sources (PDFs, CSVs, images, tables, etc.). Here we have import a simple list:

In [7]:
from edsl import ScenarioList, Scenario

In [8]:
experiences = ["Product quality", "Customer support", "Purchasing experience"]

s = ScenarioList(
    Scenario({"experience":e}) for e in experiences
)

We could also use a specific method for creating scenarios from a list:

In [9]:
s = ScenarioList.from_list("experience", experiences)

We can check the scenarios that have been created:

In [10]:
s

ScenarioList([Scenario({'experience': 'Product quality'}), Scenario({'experience': 'Customer support'}), Scenario({'experience': 'Purchasing experience'})])

To create the question variants, we pass the scenario list to the question `loop()` method, which returns a list of new questions.
We can see that each question has a new unique name and a question text with the placeholder replaced with an experience:

In [11]:
satisfied_questions = q_satisfied.loop(s)
satisfied_questions

[Question('multiple_choice', question_name = """satisfied_0""", question_text = """How satisfied are you with the following experience with our company: Product quality""", question_options = ['Extremely satisfied', 'Moderately satisfied', 'Neither satisfied nor dissatisfied', 'Moderately dissatisfied', 'Extremely dissatisfied']),
 Question('multiple_choice', question_name = """satisfied_1""", question_text = """How satisfied are you with the following experience with our company: Customer support""", question_options = ['Extremely satisfied', 'Moderately satisfied', 'Neither satisfied nor dissatisfied', 'Moderately dissatisfied', 'Extremely dissatisfied']),
 Question('multiple_choice', question_name = """satisfied_2""", question_text = """How satisfied are you with the following experience with our company: Purchasing experience""", question_options = ['Extremely satisfied', 'Moderately satisfied', 'Neither satisfied nor dissatisfied', 'Moderately dissatisfied', 'Extremely dissatisfie

We can also use the `loop()` method to create copies of the "improve" question in order to present it as a follow-up question to each of the "satisfied" questions that have been parameterized with experiences. Here, we're simply duplicating the base question without a scenario `{{ placeholder }}` because we will instead add a "memory" of the relevant "satisfied" question when administering each copy of it:

In [12]:
improve_questions = q_improve.loop(s)
improve_questions

[Question('free_text', question_name = """improve_0""", question_text = """Is there anything specific that our company can do to improve your experience?"""),
 Question('free_text', question_name = """improve_1""", question_text = """Is there anything specific that our company can do to improve your experience?"""),
 Question('free_text', question_name = """improve_2""", question_text = """Is there anything specific that our company can do to improve your experience?""")]

## Creating a survey
Next we pass a list of all the questions to a `Survey` in order to administer them together:

In [13]:
questions = [q_recommend] + satisfied_questions + improve_questions

In [14]:
from edsl import Survey

In [15]:
survey = Survey(questions)

## Adding survey logic 
In the next step we add logic to the survey specifying that each "improve" question should include a "memory" of a "satisfied" question (the question and answer that was provided):

In [16]:
for i in range(len(s)):
    survey = survey.add_targeted_memory(f"improve_{i}", f"satisfied_{i}")

We can inspect the survey details:

In [17]:
# survey

## AI agent personas
EDSL comes with a variety of methods for [designing AI agents to answer surveys](https://docs.expectedparrot.com/en/latest/agents.html).
An `Agent` is constructed by passing a dictionary of relevant `traits` with optional additional `instructions` for the language model to reference in generating responses for the agent.
Agents can be constructed from a variety of data sources, including existing survey data (e.g., a dataset of responses that were provided to some other questions). 
We can also use an EDSL question to draft some personas for agents. Here, we ask for a list of them:

In [18]:
from edsl import QuestionList

In [19]:
q_personas = QuestionList(
    question_name = "personas",
    question_text = "Draft 5 personas for diverse customers of landscaping business with varying satisfaction levels."
)

We can run this question alone and extract the response list (more on working with results below):

In [20]:
personas = q_personas.run().select("personas").to_list()[0]
personas

["Karen, a meticulous homeowner who is highly satisfied with the precision and attention to detail in her garden's upkeep",
 'Bob, a busy professional who is moderately satisfied but wishes the service was more flexible with scheduling',
 'Samantha, a new homeowner who is delighted with the transformative landscaping makeover she received',
 'Greg, an environmentally-conscious customer who is dissatisfied with the lack of sustainable options',
 'Tina, a retiree who is neutral because she appreciates the friendly service but finds it slightly overpriced']

Next we pass the personas to create a set of agents:

In [21]:
from edsl import AgentList, Agent

In [22]:
a = AgentList(
    Agent(traits = {"persona":p}) for p in personas
)

## Selecting language models
EDSL works with many popular large language models that we can select to use with a survey.
To see a list of available models:

In [23]:
from edsl import Model

In [24]:
# Model.available()

To select a model to use with a survey we pass a model name to a `Model`:

In [25]:
m = Model("gemini-pro")

If we want to compare responses for several models, we can use a `ModelList` instead:

In [26]:
from edsl import ModelList

In [27]:
m = ModelList(
    Model(model) for model in ["gemini-pro", "gpt-4o"]
)

Note: If no model is specified when running a survey, the default model GPT 4 preview is used (as above when we generated personas).

## Running a survey
We administer the survey by adding the agents and models with the `by()` method and then calling the `run()` method:

In [28]:
results = survey.by(a).by(m).run()

This generates a dataset of `Results` that includes a response for each agent/model that was used. 
We can access the results with [built-in methods for analysis](https://docs.expectedparrot.com/en/latest/results.html).
To see a list of all the components of the results:

In [29]:
# results.columns

For example, we can filter, sort and display columns of results in a table:

In [30]:
(results
 .filter("model.model == 'gemini-pro'")
 .sort_by("recommend", reverse=True)
 .select(
     "model",
     "persona",
     "recommend", "recommend_comment"
 )
 .print(format="rich")
)

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ model      ┃ agent                                     ┃ answer     ┃ comment                                   ┃
┃ .model     ┃ .persona                                  ┃ .recommend ┃ .recommend_comment                        ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemini-pro │ Samantha, a new homeowner who is          │ 10         │ The landscaping makeover completely       │
│            │ delighted with the transformative         │            │ transformed my backyard and made it a     │
│            │ landscaping makeover she received         │            │ beautiful and relaxing space for my       │
│            │                                           │            │ family and friends to enjoy. I'm so happy │
│            │                                           │            │ with the results that I've already        │
│            │                                           │            │ recommended your company to several       │
│            │                                           │            │ people.                                   │
├────────────┼───────────────────────────────────────────┼────────────┼───────────────────────────────────────────┤
│ gemini-pro │ Karen, a meticulous homeowner who is      │ 10         │ Karen is a meticulous homeowner who is    │
│            │ highly satisfied with the precision and   │            │ highly satisfied with the precision and   │
│            │ attention to detail in her garden's       │            │ attention to detail in her garden's       │
│            │ upkeep                                    │            │ upkeep. She is very likely to recommend   │
│            │                                           │            │ the company to a friend or colleague      │
│            │                                           │            │ because she is very satisfied with the    │
│            │                                           │            │ company's services.                       │
├────────────┼───────────────────────────────────────────┼────────────┼───────────────────────────────────────────┤
│ gemini-pro │ Tina, a retiree who is neutral because    │ 7          │ I appreciate the friendly service, but I  │
│            │ she appreciates the friendly service but  │            │ find the prices to be a bit high.         │
│            │ finds it slightly overpriced              │            │                                           │
├────────────┼───────────────────────────────────────────┼────────────┼───────────────────────────────────────────┤
│ gemini-pro │ Bob, a busy professional who is           │ 7          │ I'm generally happy with the service, but │
│            │ moderately satisfied but wishes the       │            │ I wish it was more flexible with          │
│            │ service was more flexible with scheduling │            │ scheduling.                               │
├────────────┼───────────────────────────────────────────┼────────────┼───────────────────────────────────────────┤
│ gemini-pro │ Greg, an environmentally-conscious        │ 2          │ I am not likely to recommend your company │
│            │ customer who is dissatisfied with the     │            │ to a friend or colleague because I am     │
│            │ lack of sustainable options               │            │ dissatisfied with the lack of sustainable │
│            │                                           │            │ options. I am an                          │
│            │                                           │            │ environmentally-conscious customer and I  │
│            │                                           │            │ believe that companies have a             │
│            │                                          

In [31]:
(results
 .filter("model.model == 'gemini-pro'")
 .sort_by("satisfied_0")
 .select("satisfied_0", "satisfied_0_comment", "improve_0")
 .print(format="rich")
)

┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ answer                  ┃ comment                                   ┃ answer                                    ┃
┃ .satisfied_0            ┃ .satisfied_0_comment                      ┃ .improve_0                                ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Extremely satisfied     │ The quality of the products used in my    │ I am absolutely thrilled with the product │
│                         │ landscaping makeover was exceptional. The │ quality and the transformative            │
│                         │ plants were healthy and vibrant, and the  │ landscaping makeover I received from your │
│                         │ materials used for the hardscaping were   │ company. The results have exceeded my     │
│                         │ durable and attractive. I am confident    │ expectations, and I couldn't be happier   │
│                         │ that my new landscape will last for many  │ with the outcome.                         │
│                         │ years to come.                            │                                           │
├─────────────────────────┼───────────────────────────────────────────┼───────────────────────────────────────────┤
│ Extremely satisfied     │ As a meticulous homeowner, I am highly    │ As a meticulous homeowner who takes great │
│                         │ satisfied with the product quality of my  │ pride in my garden, I am extremely        │
│                         │ garden's upkeep. The precision and        │ satisfied with the quality of your        │
│                         │ attention to detail are impeccable,       │ products. Your attention to detail and    │
│                         │ resulting in a stunning and vibrant       │ commitment to precision have been evident │
│                         │ outdoor space.                            │ in every interaction I've had with your   │
│                         │                                           │ company.                                  │
│                         │                                           │                                           │
│                         │                                           │ However, there is one area where I        │
│                         │                                           │ believe you could further enhance my      │
│                         │                                           │ experience:                               │
│                         │                                           │                                           │
│                         │                                           │ **Personalized Recommendations:**         │
│                         │                                           │                                           │
│                         │                                           │ I would appreciate it if you could        │
│                         │                                           │ provide personalized recommendations      │
│                         │                                           │ based on my specific gardening needs and  │
│                         │                                           │ preferences. For example, I would find it │
│                         │                                           │ helpful to receive tailored suggestions   │
│                         │                                           │ for plants that would thrive in my local  │
│                         │                                           │ climate and soil conditions, as well as   │
│                         │                                           │ tips on how to optimize their growth.     │
├─────────────────────────┼─────────────────────────────

In [32]:
(results
 .filter("model.model == 'gemini-pro'")
 .select("satisfied_1", "satisfied_1_comment", "improve_1")
 .print(pretty_labels = {
     "answer.satisfied_1": "Customer service: satisfaction",
     "comment.satisfied_1_comment": "Customer service: comment",
     "answer.improve_1": "Customer service: improvements"
 },
        format="rich")
)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Customer service: satisfaction ┃ Customer service: comment             ┃ Customer service: improvements         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Extremely satisfied            │ I am extremely satisfied with the     │ I'm absolutely thrilled with the       │
│                                │ customer support I received. The team │ exceptional customer support I've      │
│                                │ was very helpful and responsive to my │ received from your company throughout  │
│                                │ questions and concerns. They went     │ my landscaping project. The team has   │
│                                │ above and beyond to make sure I was   │ been incredibly responsive, proactive, │
│                                │ happy with my experience.             │ and attentive to my every need. I      │
│                                │                                       │ couldn't have asked for a better       │
│                                │                                       │ experience.                            │
│                                │                                       │                                        │
│                                │                                       │ However, one area where I believe      │
│                                │                                       │ there could be a slight improvement is │
│                                │                                       │ in providing more detailed updates on  │
│                                │                                       │ the progress of the project. While I   │
│                                │                                       │ appreciate the regular check-ins, I    │
│                                │                                       │ would find it even more helpful to     │
│                                │                                       │ receive more specific information      │
│                                │                                       │ about the tasks being completed and    │
│                                │                                       │ the estimated timeline for completion. │
│                                │                                       │ This would give me a clearer           │
│                                │                                       │ understanding of the project's         │
│                                │                                       │ progress and allow me to plan          │
│                                │                                       │ accordingly.                           │
├────────────────────────────────┼───────────────────────────────────────┼────────────────────────────────────────┤
│ Moderately dissatisfied        │ I am moderately dissatisfied with the │ I'm moderately dissatisfied with the   │
│                                │ customer support I received because I │ customer support I received. I would   │
│                                │ was not able to get the information I │ like to see more sustainable options   │
│                                │ needed about the company's            │ offered by your company. For example,  │
│                                │ sustainability practices. I am very   │ I would like to see more products made │
│                                │ concerned about the environment and I │ from recycled materials, and I would   │
│                                │ want to make sure that I am doing     │ like to see more options for repairing │
│                                │ business with companies that are      │ products instead of replacing them. I  │
│                                │ committed to sustaina

## Posting to the Coop
The Coop is a platform for creating, storing and sharing LLM-based research.
It is fully integrated with EDSL, allowing you to access objects from your workspace or Coop account interface.
[Learn more about creating an account and using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).
Here we post the survey and results publicly:

In [33]:
survey.push(description = "Example NPS survey", visibility = "public")

{'description': 'Example NPS survey',
 'object_type': 'survey',
 'url': 'https://www.expectedparrot.com/content/581044a0-1a97-410b-be5d-9f1b52655112',
 'uuid': '581044a0-1a97-410b-be5d-9f1b52655112',
 'version': '0.1.33.dev1',
 'visibility': 'public'}

In [34]:
results.push(description = "Results of example NPS survey", visibility = "public")

{'description': 'Results of example NPS survey',
 'object_type': 'results',
 'url': 'https://www.expectedparrot.com/content/1322aa3f-1816-41d2-815c-9417f20a2499',
 'uuid': '1322aa3f-1816-41d2-815c-9417f20a2499',
 'version': '0.1.33.dev1',
 'visibility': 'public'}

We can also post a notebook, such as this one:

In [35]:
from edsl import Notebook

In [36]:
n = Notebook(path = "nps_survey.ipynb")

In [37]:
n.push(description = "Notebook for simulating an NPS survey")

{'description': 'Notebook for simulating an NPS survey',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/0f12162f-474f-446e-9456-32b14bc87591',
 'uuid': '0f12162f-474f-446e-9456-32b14bc87591',
 'version': '0.1.33.dev1',
 'visibility': 'unlisted'}

To update an object at the Coop:

In [40]:
n = Notebook(path = "nps_survey.ipynb")

In [41]:
n.patch(uuid = "0f12162f-474f-446e-9456-32b14bc87591", visibility = "public", value = n)

{'status': 'success'}